 # Use python 2

In [1]:
import datetime
import email
import gmail 
import re
import math
import json

In [2]:
def get_body(msg):
    body = msg.get_payload(decode=True)
    char = msg.get_param('charset')
    return unicode(body, char, 'ignore').encode('utf-8')

In [3]:
 ##### setting needed !!! #####
gmail_address = "hogehoge@hogehoge"
gmail_passwd = "hogehoge"

items = [ r"アルカリイオンの水", r"サントリー　天然水（南アルプス）", r"きほんのき　紙コップ　ホット、コールド兼用" ,r'富士山系の天然水']
#items = [r"スペシャルブレンド（豆）", r"小川珈琲店 小川プレミアムブレンド （豆）"] 

n_users = 31

start = datetime.date(2017, 3, 1)
end = datetime.date(2017, 4, 1)
#######################

In [4]:
g = gmail.login(gmail_address, gmail_passwd)

In [5]:
seiyu_mails = g.inbox().mail(after=start, before=end, sender="seiyunet@the-seiyu.com")
print ("西友: " + str(len(seiyu_mails)))

alpina_mails = g.inbox().mail(after=start, before=end, sender="desk@toell.co.jp")
print ("アルピナ: "+ str(len(alpina_mails)))

西友: 1
アルピナ: 1


In [6]:
def get_seiyu(body, items):

    def get_price(target):
        try:
            print target.group()
            print '-' * 20
            price = int(target.group(3).replace(",", ""))
            price = math.ceil(price * 1.08)
            return price
        except AttributeError as e:
            print e
            return 0

    prices = []
    for item in items:
        target = re.search((item +'[\s]+価格：([\d,]+)円\(税抜\)\s+購入数量：(\d+)個\s+小計：([\d,]+)円'), body)
        prices.append(get_price(target))
    return prices

def get_alpina(body):
    target = re.search(r'合計\s+([\d,]+)円', body)
    print target.group()
    return [int(target.group(1).replace(",", ""))]

In [7]:
prices = []

for mail in seiyu_mails:
    msg = mail.fetch()
    body = get_body(msg)
    print '*' * 100
    print mail.subject
#     print mail.q
    prices.extend(get_seiyu(body, items))
#     print ("One Mail Finished")
    
#print (prices)
print ("Next Alpina")

for mail in alpina_mails:
    msg = mail.fetch()
    body = get_body(msg)
    print '*' * 100
   
    print mail.subject
    prices.extend(get_alpina(body))
#     print ("One Mail Finished")


prices.append(math.ceil(572*1.08)) #ウォータサーバ
    
print (prices)

****************************************************************************************************
[ご注文確認
アルカリイオンの水

価格：398円(税抜)
購入数量：2個
小計：796円
--------------------
サントリー　天然水（南アルプス）

価格：398円(税抜)
購入数量：2個
小計：796円
--------------------
'NoneType' object has no attribute 'group'
富士山系の天然水

価格：378円(税抜)
購入数量：2個
小計：756円
--------------------
Next Alpina
****************************************************************************************************
アルピナウォーター：ご注文ありがとうございます。
合計                                    7,161円
[860.0, 860.0, 0, 817.0, 7161, 618.0]


# 合計

In [8]:
prices += [200] #　ゴミ箱代など 

In [9]:
prices 

[860.0, 860.0, 0, 817.0, 7161, 618.0, 200]

In [10]:
price_sum = sum(prices) 
price_sum

10516.0

In [11]:
#11~2は34人
print ("ユーザ数: " + str(n_users))
print ("合計: " +str(price_sum)+  "円")
print ("一人当たり: " + str(price_sum/n_users)+  "円")

ユーザ数: 31
合計: 10516.0円
一人当たり: 339.225806452円
